In [1]:
'''
This is definition of products table.
create table public.products (
  product_id bigint generated by default as identity not null,
  created_at timestamp with time zone not null default now(),
  product_name character varying null,
  seller_id bigint not null,
  step_price double precision null,
  start_price double precision null,
  current_price double precision null,
  buy_now_price double precision null,
  price_owner_id bigint null,
  bid_turns bigint null,
  start_time timestamp with time zone null,
  end_time timestamp with time zone null,
  cat2_id bigint null,
  is_removed boolean null default false,
  description text null,
  product_images character varying[] null,
  auto_extended boolean null default false,
  fts tsvector GENERATED ALWAYS as (
    to_tsvector(
      'english'::regconfig,
      remove_accents (
        (
          (
            (COALESCE(product_name, ''::character varying))::text || ' '::text
          ) || COALESCE(description, ''::text)
        )
      )
    )
  ) STORED null,
  edited_at timestamp with time zone null,
  constraint products_pkey primary key (product_id),
  constraint products_cat2_id_fkey foreign KEY (cat2_id) references categories (id) on delete CASCADE
) TABLESPACE pg_default;

create index IF not exists products_fts on public.products using gin (fts) TABLESPACE pg_default;


'''

"\nThis is definition of products table.\ncreate table public.products (\n  product_id bigint generated by default as identity not null,\n  created_at timestamp with time zone not null default now(),\n  product_name character varying null,\n  seller_id bigint not null,\n  step_price double precision null,\n  start_price double precision null,\n  current_price double precision null,\n  buy_now_price double precision null,\n  price_owner_id bigint null,\n  bid_turns bigint null,\n  start_time timestamp with time zone null,\n  end_time timestamp with time zone null,\n  cat2_id bigint null,\n  is_removed boolean null default false,\n  description text null,\n  product_images character varying[] null,\n  auto_extended boolean null default false,\n  fts tsvector GENERATED ALWAYS as (\n    to_tsvector(\n      'english'::regconfig,\n      remove_accents (\n        (\n          (\n            (COALESCE(product_name, ''::character varying))::text || ' '::text\n          ) || COALESCE(description

# 1. Import libraries

In [2]:
import os
import json
import random
import requests
import psycopg2
from datetime import datetime, timedelta
from dotenv import load_dotenv
from bs4 import BeautifulSoup
# No need to translate because this notebook is for api using Vietnamese language

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}


# Define category from our database
CATEGORY_ID_DB = None
CATEGORY_LIST_DB = []

LIMIT = 20
PAGE = 1
MAX_PRODUCTS = 20


TIKI_CATEGORY_URL = "https://tiki.vn/api/personalish/v1/blocks/listings" # query params: limit, category
TIKI_PRODUCT_URL = "https://tiki.vn/api/v2/products/" # params: product_id, query params: include, platform, spider

In [70]:
# Define category from Tiki API
CATEGORY_LIST_API = [
    {
        "id": 925,
        "name": "Thời Trang Nam"
    },
    {
        "id": 5404,
        "name": "Thời Trang Nữ"
    },
    {
        "id": 27572,
        "name": "Giày Dép"
    },
    {
        "id": 1794,
        "name": "Máy Tính Bảng"
    },
    {
        "id": 1795,
        "name": "Điện Thoại"
    },
    {
        "id": 8095,
        "name": "Laptop"
    },
    {
        "id": 1815,
        "name": "Phụ Kiện Số"
    },
    {
        "id": 1801,
        "name": "Máy Ảnh"
    },
    {
        "id": 27224,
        "name": "Tranh Sơn Dầu"
    },
    {
        "id": 23102,
        "name": "Linh Vật May Mắn"
    },
    {
        "id": 8374,
        "name": "Trang Sức"
    }
]
CATEGORY_ID_API = None

In [75]:
def load_subcategories_from_db():
    """Load all subcategories from database"""
    load_dotenv()
    connection_string = os.getenv("SUPABASE_CONNECTION_STRING")
    
    conn = psycopg2.connect(connection_string)
    cursor = conn.cursor()
    
    try:
        # Fetch all subcategories (categories with parent_id not null)
        cursor.execute(
            """
            SELECT id, name 
            FROM categories 
            WHERE parent_id IS NOT NULL 
            ORDER BY id
            """
        )
        
        rows = cursor.fetchall()
        category_list = [{"id": row[0], "name": row[1]} for row in rows]
        
        print(f"Loaded {len(category_list)} subcategories from database")
        print("Sample categories:", category_list[:3])
        
        return category_list
        
    except Exception as e:
        print(f"Error: {e}")
        return []
        
    finally:
        cursor.close()
        conn.close()

# Load subcategories into CATEGORY_LIST_DB
CATEGORY_LIST_DB = load_subcategories_from_db()

Loaded 10 subcategories from database
Sample categories: [{'id': 4, 'name': 'Thời Trang Nam'}, {'id': 5, 'name': 'Thời Trang Nữ'}, {'id': 6, 'name': 'Giày Dép'}]


In [6]:
def generate_start_time():
    """Generate random start_time from 3 days before to 3 days after today"""
    now = datetime.now()
    days_offset = random.randint(-3, 3)
    hours_offset = random.randint(0, 23)
    start_time = now + timedelta(days=days_offset, hours=hours_offset)
    return start_time

def generate_end_time(start_time):
    """Generate end_time that is 1-20 days after start_time"""
    days_to_add = random.randint(1, 20)
    end_time = start_time + timedelta(days=days_to_add)
    return end_time

def generate_prices():
    """Generate các giá theo mệnh giá Việt Nam"""
    start_price = random.randint(50, 5000) * 10000
    step_price = random.randint(10, 200) * 10000
    buy_now_price = start_price + random.randint(500, 2000) * 10000
    
    return {
        'start_price': start_price,
        'current_price': start_price,
        'step_price': step_price,
        'buy_now_price': buy_now_price
    }

def get_seller_ids(cursor):
    """Lấy tất cả seller_ids từ bảng users"""
    cursor.execute("SELECT user_id FROM users ORDER BY user_id")
    seller_ids = [row[0] for row in cursor.fetchall()]
    
    return seller_ids

def distribute_seller_ids(total_products, seller_ids):

    result = []
    
    """Phân bổ seller_ids hoàn toàn random"""
    result = [random.choice(seller_ids) for _ in range(total_products)]
    return result



In [7]:
def is_valid_category_id_api ():
    """Check if the CATEGORY_ID_API is valid"""
    global CATEGORY_ID_API
    for category in CATEGORY_LIST_API:
        if category['id'] == CATEGORY_ID_API:
            return True
    return False

def is_valid_category_id_db ():
    """Check if the CATEGORY_ID_DB is valid"""
    global CATEGORY_ID_DB
    for category in CATEGORY_LIST_DB:
        if category['id'] == CATEGORY_ID_DB:
            return True
    return False

In [8]:
def collect_product_images(product_detail, min_images=3, max_images=10):
    """
    Collect product images from product_detail data
    
    Args:
        product_detail: Product detail from API response
        min_images: Minimum number of images required (default: 3)
        max_images: Maximum number of images to collect (default: 10)
    
    Returns:
        list: List of image URLs (3-10 images)
    """
    image_urls = []
    
    # 1. Get base_url from images array
    images = product_detail.get('images', [])
    if images and 'base_url' in images[0]:
        base_url = images[0].get('base_url')
        if base_url:
            image_urls.append(base_url)
    
    # 2. Get images from configurable_products (more diverse collection)
    configurable_products = product_detail.get('configurable_products', [])
    if configurable_products:
        for config_product in configurable_products:
            # Check if we already have enough images
            if len(image_urls) >= max_images:
                break
            
            # Get images array from each configurable product
            config_images = config_product.get('images', [])
            if config_images:
                # Randomly decide how many images to take from this config (1-3)
                num_to_take = min(random.randint(1, 3), len(config_images))
                
                # Randomly select which images to take
                if len(config_images) > num_to_take:
                    selected_indices = random.sample(range(len(config_images)), num_to_take)
                else:
                    selected_indices = range(len(config_images))
                
                # Extract large_url from selected images
                for idx in selected_indices:
                    if len(image_urls) >= max_images:
                        break
                    
                    large_url = config_images[idx].get('large_url')
                    if large_url and large_url not in image_urls:
                        image_urls.append(large_url)
    
    # 3. If not enough images, fill with base_url
    if len(image_urls) < min_images and images and 'base_url' in images[0]:
        base_url = images[0].get('base_url')
        while len(image_urls) < min_images and base_url:
            if base_url not in image_urls:
                image_urls.append(base_url)
            else:
                # Add with a marker to ensure we have at least min_images
                image_urls.append(base_url)
                break
    
    # 4. Ensure we have at least min_images
    while len(image_urls) < min_images:
        # If still not enough, use first available or empty string
        if image_urls:
            image_urls.append(image_urls[0])
        else:
            # Fallback: add placeholder if no images at all
            image_urls.append('')
    
    # 5. Limit to max_images
    return image_urls[:max_images]

In [71]:
# Step 1: Display available categories
print("=== TIKI API CATEGORIES ===")
if CATEGORY_LIST_API:
    for idx, cat in enumerate(CATEGORY_LIST_API, 1):
        print(f"{idx}. ID: {cat['id']}, Name: {cat['name']}")
else:
    print("CATEGORY_LIST_API is empty. Please add categories manually.")

print("\n=== DATABASE CATEGORIES (Parent Categories) ===")
for idx, cat in enumerate(CATEGORY_LIST_DB, 1):
    print(f"{idx}. ID: {cat['id']}, Name: {cat['name']}")

# Step 2: Select categories
CATEGORY_ID_API = int(input("\nEnter Tiki API Category ID: "))
CATEGORY_ID_DB = int(input("Enter Database Category ID: "))

# Step 3: Validate selected categories
if not is_valid_category_id_api():
    print(f"Error: Invalid Tiki API Category ID: {CATEGORY_ID_API}")
    exit(1)
if not is_valid_category_id_db():
    print(f"Error: Invalid Database Category ID: {CATEGORY_ID_DB}")
    exit(1)

print(f"\nSelected API Category: {CATEGORY_ID_API}")
print(f"Selected DB Category: {CATEGORY_ID_DB}")

=== TIKI API CATEGORIES ===
1. ID: 925, Name: Thời Trang Nam
2. ID: 5404, Name: Thời Trang Nữ
3. ID: 27572, Name: Giày Dép
4. ID: 1794, Name: Máy Tính Bảng
5. ID: 1795, Name: Điện Thoại
6. ID: 8095, Name: Laptop
7. ID: 1815, Name: Phụ Kiện Số
8. ID: 1801, Name: Máy Ảnh
9. ID: 27224, Name: Tranh Sơn Dầu
10. ID: 23102, Name: Linh Vật May Mắn
11. ID: 8374, Name: Trang Sức

=== DATABASE CATEGORIES (Parent Categories) ===
1. ID: 4, Name: Thời Trang Nam
2. ID: 5, Name: Thời Trang Nữ
3. ID: 6, Name: Giày Dép
4. ID: 7, Name: Điện Thoại & Máy Tính Bảng
5. ID: 8, Name: Laptop & PC
6. ID: 9, Name: Phụ Kiện Số
7. ID: 10, Name: Máy Ảnh & Quay Phim
8. ID: 11, Name: Tranh Ảnh
9. ID: 12, Name: Tượng & Điêu Khắc
10. ID: 13, Name: Đồ Cổ & Hiếm

Selected API Category: 8374
Selected DB Category: 13


In [72]:
# Step 3: Fetch products from Tiki API and generate JSON data
products_data = []

# Input limit and page
print("\n=== FETCH CONFIGURATION ===")
LIMIT = int(input(f"Enter limit (current: {LIMIT}): ") or LIMIT)
PAGE = int(input(f"Enter page (current: {PAGE}): ") or PAGE)
MAX_PRODUCTS = int(input(f"Enter max products to fetch (current: {MAX_PRODUCTS}): ") or MAX_PRODUCTS)

# Fetch product list from category
print(f"\nFetching products from category {CATEGORY_ID_API}...")
print(f"Parameters: LIMIT={LIMIT}, PAGE={PAGE}, MAX_PRODUCTS={MAX_PRODUCTS}")
category_response = requests.get(
    TIKI_CATEGORY_URL,
    headers=headers,
    params={'limit': LIMIT, 'category': CATEGORY_ID_API, 'page': PAGE}
)
print (f"Category fetch status: {category_response.status_code}")
if category_response.status_code == 200:
    category_data = category_response.json()
    print("API response data:", category_data)
    product_list = []
    
    # Extract products from response - data is array of product objects
    if 'data' in category_data:
        product_list = category_data['data']
    
    print(f"Found {len(product_list)} products")
    
    # Connect to database to get seller IDs
    load_dotenv()
    conn = psycopg2.connect(os.getenv("SUPABASE_CONNECTION_STRING"))
    cursor = conn.cursor()
    
    seller_ids = get_seller_ids(cursor)
    distributed_sellers = distribute_seller_ids(min(len(product_list), MAX_PRODUCTS), seller_ids)
    
    # Process each product
    for idx, product in enumerate(product_list[:MAX_PRODUCTS]):
        product_id = product.get('id')
        print(f"Processing product {idx + 1}/{min(len(product_list), MAX_PRODUCTS)}: ID {product_id}")
        
        # Fetch detailed product info
        product_response = requests.get(
            f"{TIKI_PRODUCT_URL}{product_id}",
            headers=headers,
            params={'platform': 'web', 'spid': product_id}
        )
        
        if product_response.status_code == 200:
            product_detail = product_response.json()
            
            # Extract product information
            product_name = product_detail.get('name', '')
            description = product_detail.get('description', '')
            images = collect_product_images(product_detail, min_images=3, max_images=10)
            
            # Generate auction data
            prices = generate_prices()
            start_time = generate_start_time()
            end_time = generate_end_time(start_time)
            seller_id = distributed_sellers[idx]
            
            # Prepare row data
            products_data.append({
                'product_name': product_name,
                'seller_id': seller_id,
                'step_price': prices['step_price'],
                'start_price': prices['start_price'],
                'current_price': prices['current_price'],
                'buy_now_price': prices['buy_now_price'],
                'start_time': start_time.isoformat(),
                'end_time': end_time.isoformat(),
                'cat2_id': CATEGORY_ID_DB,
                'description': description,
                'product_images': images,
                'auto_extended': random.choice([True, False])
            })
        else:
            print(f"Failed to fetch product {product_id}")
    
    cursor.close()
    conn.close()
    
    # Step 4: Write to JSON file
    json_file_path = 'product.api.json'
    with open(json_file_path, 'w', encoding='utf-8') as jsonfile:
        json.dump(products_data, jsonfile, ensure_ascii=False, indent=2)
    
    print(f"\nSuccessfully exported {len(products_data)} products to {json_file_path}")
else:
    print(f"Failed to fetch category products: {category_response.status_code}")


=== FETCH CONFIGURATION ===



Fetching products from category 8374...
Parameters: LIMIT=30, PAGE=1, MAX_PRODUCTS=30
Category fetch status: 200
API response data: {'block': {'code': 'tiki_listing', 'title': 'Tiki Listing', 'icon': ''}, 'data': [{'id': 277425868, 'sku': '9181155253300', 'name': 'Dây Chuyền Nam Bạc Ta Họa Tiết Đốt Rồng Bạc Hiểu Minh DBN085', 'url_key': 'day-chuyen-nam-bac-ta-hoa-tiet-dot-rong-bac-hieu-minh-dbn085-p277425868', 'url_path': 'day-chuyen-nam-bac-ta-hoa-tiet-dot-rong-bac-hieu-minh-dbn085-p277425868.html?spid=277425869', 'type': '', 'author_name': '', 'book_cover': None, 'brand_name': 'Bạc Hiểu Minh', 'short_description': '', 'price': 2189000, 'list_price': 0, 'badges': [], 'badges_new': [{'code': 'authentic_brand', 'icon': 'https://salt.tikicdn.com/ts/tka/69/cf/22/1be823299ae34c7ddcd922e73abd4909.png', 'icon_height': 20, 'icon_width': 89, 'placement': 'under_product_image', 'type': 'icon_badge'}, {'code': 'video_icon', 'icon': 'https://salt.tikicdn.com/ts/upload/d5/f3/6f/fdd40db52ff4afe497

In [73]:
# Step 5: Generate SQL INSERT statements from JSON
print("\n=== Generating SQL INSERT statements ===")

# Read JSON file
with open('product.api.json', 'r', encoding='utf-8') as f:
    products = json.load(f)

# Generate SQL INSERT statements
sql_statements = []
sql_statements.append("-- Insert products from Tiki API\n")

for product in products:
    # Escape single quotes in strings
    product_name = product['product_name'].replace("'", "''")
    description = product['description'].replace("'", "''") if product['description'] else ''
    
    # Format product_images array for PostgreSQL
    images_array = "ARRAY[" + ", ".join([f"'{img}'" for img in product['product_images']]) + "]"
    
    # Generate INSERT statement
    sql = f"""INSERT INTO products (product_name, seller_id, step_price, start_price, current_price, buy_now_price, start_time, end_time, cat2_id, description, product_images, auto_extended)
VALUES ('{product_name}', {product['seller_id']}, {product['step_price']}, {product['start_price']}, {product['current_price']}, {product['buy_now_price']}, '{product['start_time']}', '{product['end_time']}', {product['cat2_id']}, '{description}', {images_array}, {str(product['auto_extended']).lower()});
"""
    sql_statements.append(sql)

# Write to SQL file
sql_file_path = 'product.insert.sql'
with open(sql_file_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(sql_statements))

print(f"Successfully generated SQL file: {sql_file_path}")
print(f"Total INSERT statements: {len(products)}")


=== Generating SQL INSERT statements ===
Successfully generated SQL file: product.insert.sql
Total INSERT statements: 30


In [74]:
# Step 6: Execute SQL file to insert products into database
print("\n=== Executing SQL INSERT statements ===")

# Read SQL file
with open('product.insert.sql', 'r', encoding='utf-8') as f:
    sql_content = f.read()

# Connect to database and execute
load_dotenv()
conn = psycopg2.connect(os.getenv("SUPABASE_CONNECTION_STRING"))
cursor = conn.cursor()

try:
    # Execute SQL statements
    cursor.execute(sql_content)
    conn.commit()
    
    # Get inserted count
    cursor.execute("SELECT COUNT(*) FROM products")
    total_count = cursor.fetchone()[0]
    
    print(f"✓ Successfully inserted products into database")
    print(f"Total products in database: {total_count}")
    
except Exception as e:
    conn.rollback()
    print(f"✗ Error executing SQL: {e}")
    
finally:
    cursor.close()
    conn.close()


=== Executing SQL INSERT statements ===
✓ Successfully inserted products into database
Total products in database: 310
